In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('../res/AirQualityUCI/AirQualityUCI.csv', sep=';', na_values=-200)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9471 entries, 0 to 9470
Data columns (total 17 columns):
Date             9357 non-null object
Time             9357 non-null object
CO(GT)           7765 non-null object
PT08.S1(CO)      8991 non-null float64
NMHC(GT)         914 non-null float64
C6H6(GT)         9357 non-null object
PT08.S2(NMHC)    8991 non-null float64
NOx(GT)          7718 non-null float64
PT08.S3(NOx)     8991 non-null float64
NO2(GT)          7715 non-null float64
PT08.S4(NO2)     8991 non-null float64
PT08.S5(O3)      8991 non-null float64
T                8991 non-null object
RH               8991 non-null object
AH               8991 non-null object
Unnamed: 15      0 non-null float64
Unnamed: 16      0 non-null float64
dtypes: float64(10), object(7)
memory usage: 1.2+ MB
None


,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578",NaN,NaN
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255",NaN,NaN
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502",NaN,NaN
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867",NaN,NaN
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888",NaN,NaN


In [4]:
names = ['date',
 'time',
 'co_gt',
 'pt08_s1_co',
 'nmhc_gt',
 'c6h6_gt',
 'pt08_s2_nmhc',
 'nox_gt',
 'pt08_s3_nox',
 'no2_gt',
 'pt08_s4_no2',
 'pt08_s5_o3',
 't',
 'rh',
 'ah',
 'Unnamed_1',
 'Unnamed_2']
df = df.rename(columns= {last_key: new_key for last_key, new_key in zip(df.keys(), names)})


стоит убрать столбец `nmhc_gt` так как он не несет информативности и имеет пропуски в 90% случаях 

In [5]:
df = df.drop(columns=['Unnamed_1', 'Unnamed_2', 'nmhc_gt'])
df = df.dropna(thresh=len(df.keys()))

In [6]:
df['date'] = pd.to_datetime(df.date, format='%d/%m/%Y')
df['time'] = pd.to_datetime(df.time, format='%H.%M.%S')
df.co_gt = pd.to_numeric(df.co_gt.str.replace(',', '.'))
df.c6h6_gt = pd.to_numeric(df.c6h6_gt.str.replace(',', '.'))
df.t = pd.to_numeric(df.t.str.replace(',', '.'))
df.rh = pd.to_numeric(df.rh.str.replace(',', '.'))
df.ah = pd.to_numeric(df.ah.str.replace(',', '.'))
df.info()
df.shape[0] - df.dropna().count()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7024 entries, 0 to 9356
Data columns (total 14 columns):
date            7024 non-null datetime64[ns]
time            7024 non-null datetime64[ns]
co_gt           7024 non-null float64
pt08_s1_co      7024 non-null float64
c6h6_gt         7024 non-null float64
pt08_s2_nmhc    7024 non-null float64
nox_gt          7024 non-null float64
pt08_s3_nox     7024 non-null float64
no2_gt          7024 non-null float64
pt08_s4_no2     7024 non-null float64
pt08_s5_o3      7024 non-null float64
t               7024 non-null float64
rh              7024 non-null float64
ah              7024 non-null float64
dtypes: datetime64[ns](2), float64(12)
memory usage: 823.1 KB


date            0
time            0
co_gt           0
pt08_s1_co      0
c6h6_gt         0
pt08_s2_nmhc    0
nox_gt          0
pt08_s3_nox     0
no2_gt          0
pt08_s4_no2     0
pt08_s5_o3      0
t               0
rh              0
ah              0
dtype: int64

In [7]:
import seaborn as sns
sns.heatmap(df.corr(), annot= True)

In [8]:
corr = pd.DataFrame(df.corr().c6h6_gt.drop('c6h6_gt'))
corr[(corr.c6h6_gt > np.abs(0.5))]

,c6h6_gt
pt08_s1_co,0.878285
pt08_s2_nmhc,0.982213
nox_gt,0.719871
no2_gt,0.604771
pt08_s4_no2,0.764879
pt08_s5_o3,0.861747


In [9]:
from sklearn.preprocessing import StandardScaler     
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [19]:
# x = StandardScaler().fit_transform(df.drop(columns=['c6h6_gt', 'date', 'time']))
x = StandardScaler().fit_transform(df[list(corr[(corr.c6h6_gt > np.abs(0.5))].index)])
y = df.c6h6_gt
x_train, x_test, y_train, y_test=train_test_split(x ,y,test_size=0.3, random_state=42)

lr=LinearRegression()
lr_model=lr.fit(x_train,y_train)

y_pred=lr_model.predict(x_test)
rmse=np.sqrt(mean_squared_error(y_test,y_pred))
print('RMSE of Linear model:',rmse)

RMSE of Linear model: 1.3407886388772874


In [20]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import GridSearchCV 
model = DecisionTreeRegressor()

In [21]:
model=model.fit(x_train,y_train)
y_pred = model.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_pred,y_test))
print('RMSE of Decision Tree Regression:', rmse)

RMSE of Decision Tree Regression: 0.2575243397792676


In [26]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, 

def create_polynomial_regression_model(x, y, degree):
    x_train, x_test, y_train, y_test=train_test_split(x ,y,test_size=0.3, random_state=42)
    poly_features = PolynomialFeatures(degree=degree)
    x_train_poly = poly_features.fit_transform(x_train)
    x_test_poly = poly_features.fit_transform(x_test)

    poly_model = LinearRegression()
    poly_model.fit(x_train_poly, y_train)


    y_train_predicted = poly_model.predict(x_train_poly)
    y_test_predict = poly_model.predict(x_test_poly)
    
    
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_predicted))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_predict))
    
    
    r2_train = r2_score(y_train, y_train_predicted)    
    r2_test = r2_score(y_test, y_test_predict)

    
    print("RMSE of training set is {}".format(rmse_train))
    print("R2 score of training set is {}".format(r2_train))
    print("RMSE of test set is {}".format(rmse_test))
    print("R2 score of test set is {}".format(r2_test))


for i in range(2, 4):
    print('Degree - ', i)
    create_polynomial_regression_model(x, y, i)
    print("\n")

Degree -  2
RMSE of training set is 0.06226608107991607
R2 score of training set is 0.9999314590642818
RMSE of test set is 0.07212303250251242
R2 score of test set is 0.9999038108934262


Degree -  3
RMSE of training set is 0.030674401281807654
R2 score of training set is 0.9999833659059433
RMSE of test set is 0.03132435349950685
R2 score of test set is 0.9999818556186149


